In [1]:
import os
import cv2 
import time 
import sys
import argparse 
import multiprocessing 
import numpy as np 
import tensorflow as tf 
from matplotlib import pyplot as plt
import matplotlib # Matplotlib chooses Xwindows backend by default. matplotlib.use('Agg') 
from object_detection.utils import label_map_util 
from object_detection.utils import visualization_utils as vis_util


In [2]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_2018_01_28' 
PATH_TO_CKPT = os.path.join(MODEL_NAME, 'frozen_inference_graph.pb') # List of the strings that is used to add correct label for each box. 
PATH_TO_LABELS = os.path.join('data', 'final_label_map.pbtxt')
NUM_CLASSES = 2 
label_map = label_map_util.load_labelmap(PATH_TO_LABELS) 
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True) 
category_index = label_map_util.create_category_index(categories)


In [3]:
def detect_objects(image_np, sess, detection_graph): 
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3] 
    image_np_expanded = np.expand_dims(image_np, axis=0)
    #loader = tf.train.import_meta_graph(model_path+'.meta')
    #loader.restore(sess,model_path)
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

    scores = detection_graph.get_tensor_by_name('detection_scores:0')
    classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    (boxes, scores, classes, num_detections) = sess.run(
        [boxes, scores, classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})

    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np, 
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32), 
        np.squeeze(scores),
        category_index, 
        use_normalized_coordinates=True,
        line_thickness=8)

    return image_np



In [4]:
#Load a frozen TF model 
detection_graph = tf.Graph() 
with detection_graph.as_default(): 
    od_graph_def = tf.GraphDef() 
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid: 
        serialized_graph = fid.read() 
        od_graph_def.ParseFromString(serialized_graph) 
        tf.import_graph_def(od_graph_def, name='')


In [5]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [6]:
 def process_image(image):
        # NOTE: The output you return should be a color image (3 channel) for processing video below 
        # you should return the final output (image with lines are drawn on lanes) 
        with detection_graph.as_default(): 
            with tf.Session(graph=detection_graph) as sess: 
                image_process = detect_objects(image, sess, detection_graph) 
                return image_process


In [7]:
# 1.使用 VideoFileClip 函数从视频中抓取图片。 
# 2.用fl_image函数将原图片替换为修改后的图片，用于传递物体识别的每张抓取图片。 
# 3.所有修改的剪辑图像被组合成为一个新的视频。

white_output = 'video1_out.mp4'
clip1 = VideoFileClip("video1_clip.mp4").subclip(5,8)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!s
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video video1_out.mp4
[MoviePy] Writing video video1_out.mp4


 97%|███████████████████████████████████████████████████████████████████████████████▊  | 36/37 [02:50<00:04,  4.95s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video1_out.mp4 

Wall time: 2min 51s


In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [9]:
from moviepy.editor import *
#clip1 = VideoFileClip("video1_out.mp4")
#clip1.write_gif("final.gif")